In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
from matplotlib import pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1- Get Data

In [ ]:
!apt-get install p7zip
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/train.tsv.7z
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/test.tsv.7z
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/sample_submission.csv.7z

In [ ]:
!unzip /kaggle/input/mercari-price-suggestion-challenge/sample_submission_stg2.csv.zip
!unzip /kaggle/input/mercari-price-suggestion-challenge/test_stg2.tsv.zip

In [ ]:
train = pd.read_csv('train.tsv', sep = '\t')
test = pd.read_csv('test_stg2.tsv', sep='\t')

In [ ]:
print ("Train data shape:", train.shape)
print ("Test data shape:", test.shape)

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
train.describe()

# 2- Data Preprocessing

> # 1) Missing data

In [ ]:
full_data = pd.concat([train, test], sort=False)

In [ ]:
full_data.isnull().sum()

In [ ]:
print(pd.isnull(train).sum())
print("------------")
print(pd.isnull(test).sum())

In [ ]:
train.brand_name.fillna(value = "NoBrand", inplace = True)
test.brand_name.fillna(value = "NoBrand", inplace = True)

train.category_name.fillna(value = "Other/Other/Other", inplace = True)
test.category_name.fillna(value = "Other/Other/Other", inplace = True)

train.item_description.fillna(value = "No description yet", inplace = True)
test.item_description.fillna(value = "No description yet", inplace = True)

In [ ]:
print(train.isnull().any())
print("------------")
print(test.isnull().any())

> # 2) Subclasses

In [ ]:
def split(txt):
    try :
        return txt.split("/")
    except :
        return ("Other", "Other", "Other")


In [ ]:
train['general_category']='' 
train['subcategory_1'] = '' 
train['subcategory_2'] = ''

test['general_category']='' 
test['subcategory_1'] = '' 
test['subcategory_2'] = ''


In [ ]:
train['general_category'],train['subcategory_1'],train['subcategory_2'] = \
zip(*train['category_name'].apply(lambda x: split(x)))

test['general_category'],test['subcategory_1'],test['subcategory_2'] = \
zip(*test['category_name'].apply(lambda x: split(x)))


In [ ]:
train

In [ ]:
train.head()

In [ ]:
train.price.plot.hist(bins=50, figsize=(8,4), edgecolor='white',range=[0,300])

plt.title('Price Distribution')

> # 3) EDA

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
train.hist(bins=50, figsize=(20,15))
plt.show()

In [ ]:
train['item_description'].value_counts()

In [ ]:
train['item_condition_id'].value_counts().plot(kind='bar')

In [ ]:
train['brand_name'].value_counts()

In [ ]:
train['general_category'].value_counts().plot(kind='bar')

In [ ]:
train['subcategory_1'].value_counts().plot(kind='bar')
train['subcategory_1'].value_counts()

In [ ]:
train['subcategory_2'].value_counts().plot(kind='bar')
train['subcategory_2'].value_counts()

In [ ]:
log_prices = np.log1p(train.price)
log_prices.hist()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train_item_condition_id = scaler.fit_transform(train['item_condition_id'].values.reshape(-1,1))

In [ ]:
from sklearn.preprocessing import LabelBinarizer

lb_brand_name = LabelBinarizer(sparse_output=True)
X_train_brand = lb_brand_name.fit_transform(train['brand_name'])

lb_shipping = LabelBinarizer(sparse_output=True)
X_train_shipping = lb_shipping.fit_transform(train['shipping'])

lb_cat_0 = LabelBinarizer(sparse_output=True)
X_train_general_category = lb_cat_0.fit_transform(train['general_category'])

lb_cat_1 = LabelBinarizer(sparse_output=True)
X_train_subcategory_1 = lb_cat_1.fit_transform(train['subcategory_1'])

lb_cat_2 = LabelBinarizer(sparse_output=True)
X_train_subcategory_2 = lb_cat_2.fit_transform(train['subcategory_2'])


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

tfidf_desc = TfidfVectorizer(max_features=50000, ngram_range=(1, 1), stop_words='english', norm='l2',lowercase=True)
name_vectorizer = CountVectorizer(stop_words='english')

X_train_descp = tfidf_desc.fit_transform(train['item_description'])
X_train_name = name_vectorizer.fit_transform(train['name'])


In [ ]:
from scipy.sparse import hstack

X_train = hstack((X_train_name, X_train_descp, X_train_brand, X_train_item_condition_id, X_train_shipping, X_train_general_category, X_train_subcategory_1, X_train_subcategory_2)).tocsr()


In [ ]:
X_train

**The correlation with price**

In [ ]:
corr_matrix = train.corr()
corr_matrix

In [ ]:
from scipy.sparse import hstack
X_train = hstack((X_train_name, X_train_descp, X_train_brand, X_train_item_condition_id, 
X_train_shipping, X_train_general_category, X_train_subcategory_1, X_train_subcategory_2)).tocsr()

In [ ]:
import seaborn as sns

sns.heatmap(corr_matrix, 
            xticklabels=corr_matrix.columns.values,
            yticklabels=corr_matrix.columns.values)

# 3- Stratified sampling


In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=42)
for train_index, test_index in split.split(train, train['general_category']):
    train_idx = train_index
    test_idx = test_index

In [ ]:
test_data = X_train[test_idx]
log_test_prices = log_prices.iloc[test_idx]
train_data = X_train[train_idx]
log_train_prices = log_prices.iloc[train_idx]

# 4- Obtain selected models

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

def performance(ytrue, ypredicted):
    print("MSE: ",np.sqrt(mean_squared_error(ytrue, ypredicted))) 
    print("MAE: ", np.sqrt(mean_absolute_error(ytrue, ypredicted)))

> **1) Linear Regression**

In [ ]:
#from sklearn.linear_model import LinearRegression 
#regressor = LinearRegression() 
#regressor.fit(train_data, log_train_prices)

In [ ]:
#y_pred = regressor.predict(test_data)

In [ ]:
#from sklearn.metrics import mean_absolute_error
#from sklearn.metrics import mean_squared_error
    
#performance(log_test_prices, y_pred)

> **2) SGD Regressor**

In [ ]:
from sklearn.linear_model import SGDRegressor

# SkLearn SGD classifier
clf_ = SGDRegressor()
clf_.fit(train_data, log_train_prices)
performance(log_test_prices, clf_.predict(test_data))

> **3) Ridge & Lasso**

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, \
                                 RidgeCV, Lasso, LassoCV, \
                                 ElasticNet, ElasticNetCV

In [ ]:
#lasso = Lasso(max_iter = 100000, normalize = True)

#lassocv = LassoCV(alphas = None, cv = 10, max_iter = 100000, normalize = True)
#lassocv.fit(train_data, log_train_prices)

#lasso.set_params(alpha=lassocv.alpha_)
#lasso.fit(train_data, log_train_prices)

#print('The Lasso I:')
#print("Alpha =", lassocv.alpha_)
#performance(log_test_prices, lasso.predict(test_data))


In [ ]:
ridge_lin_reg = Ridge(alpha=4, max_iter= 500)
ridge_lin_reg.fit(train_data, log_train_prices)

performance(log_test_prices,ridge_lin_reg.predict(test_data))

> **4) SVR**

In [ ]:
from sklearn.svm import SVR

kernel_SVR = SVR(kernel='rbf', verbose=True, max_iter= 1000)
kernel_SVR.fit(train_data, log_train_prices)

performance(log_test_prices,kernel_SVR.predict(test_data))

> **5) Random forest**

**I took MSE From Colab Note book because it took too much time to run Same also in LGBM regressor**

In [ ]:
MSE:  0.6012809577308371
MAE:  0.6765551529927671

In [ ]:

#from sklearn.ensemble import RandomForestRegressor

#forest_reg = RandomForestRegressor(verbose=True, max_depth=15, n_estimators=30)
#forest_reg.fit(train_data, log_train_prices)

#performance(log_test_prices, forest_reg.predict(test_data))

> **6) LGBM Regressor**

In [ ]:
MSE:  0.59995163968476
MAE:  0.6796141406429075

In [ ]:
#from lightgbm import LGBMRegressor

In [ ]:

#lgbm_model = LGBMRegressor(verbose=0, max_depth=15, n_estimators=30)
#lgbm_model.fit(train_data, log_train_prices, verbose=0)

#performance(log_test_prices,lgbm_model.predict(test_data))

# 5- Fine-tune model

 **While LGBM has the least of error, SO need to fine-tune it**

In [ ]:
from sklearn.model_selection import PredefinedSplit


In [ ]:
split_index = [-1 if x in train_idx else 0 for x in range(X_train.shape[0])]


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distribs = {
        'n_estimators': randint(low=5, high=200),
        'max_depth': randint(low=10, high=100),
    }

lgbm_srch = LGBMRegressor()
pds = PredefinedSplit(test_fold = split_index)

rnd_search = RandomizedSearchCV(lgbm_srch, param_distributions=param_distribs,
                                n_iter=20, cv=pds, scoring='neg_mean_squared_error')

rnd_search.fit(X_train, log_prices)
performance(log_test_prices,rnd_search.best_estimator_.predict(test_data))

> ****Linear Regression, Lasso, Random forest, and LGBM Regressor took too much time to run ****